In [ ]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import xgboost as xgb

In [ ]:
data = pd.read_csv('data.csv')
data.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [ ]:
# proses case folding
import re

def casefolding(Sentence):
    Sentence = Sentence.lower()
    Sentence = Sentence.strip(" ")
    Sentence = re.sub(r'[?|$|.|!_:")(-+,]', '', Sentence)
    return Sentence
data['Sentence'] = data['Sentence'].apply(casefolding)
data.head(100)

,Sentence,Sentiment
0,the geosolutions technology will leverage bene...,positive
1,esi on lows down 150 to 250 bk a real possibility,negative
2,for the last quarter of 2010 componenta 's ne...,positive
3,according to the finnish-russian chamber of co...,neutral
4,the swedish buyout firm has sold its remaining...,neutral
...,...,...
95,m-real said there are ` no grounds ' for the r...,neutral
96,lember said the matter was topical also in est...,neutral
97,- moody 's said it gave p a1 long-term senior ...,neutral
98,tsla recall,negative


In [ ]:
def token(sentence):
    # Memisahkan kalimat menjadi kata-kata menggunakan spasi sebagai pemisah
    nstr = sentence.split(' ')

    # Membuat list untuk menyimpan indeks kata yang akan dihapus
    indices_to_remove = []

    # Mengidentifikasi indeks kata yang akan dihapus (misalnya, string kosong)
    for i, word in enumerate(nstr):
        if word == '':
            indices_to_remove.append(i)

    # Menghapus kata berdasarkan indeks yang telah diidentifikasi
    for index in reversed(indices_to_remove):
        del nstr[index]

    return nstr

# Contoh penggunaan:
data['Sentence'] = data['Sentence'].apply(token)
data.head(10)

,Sentence,Sentiment
0,"[the, geosolutions, technology, will, leverage...",positive
1,"[esi, on, lows, down, 150, to, 250, bk, a, rea...",negative
2,"[for, the, last, quarter, of, 2010, componenta...",positive
3,"[according, to, the, finnish-russian, chamber,...",neutral
4,"[the, swedish, buyout, firm, has, sold, its, r...",neutral
5,"[spy, wouldn't, be, surprised, to, see, a, gre...",positive
6,"[shell's, 70, billion, bg, deal, meets, shareh...",negative
7,"[ssh, communications, security, corp, stock, e...",negative
8,"[kone, 's, net, sales, rose, by, some, 14, %, ...",positive
9,"[the, stockmann, department, store, will, have...",neutral


In [ ]:
# proses filtering

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def stopword_removal(comments):
    filtering = stopwords.words('indonesian', 'english')
    x = []
    data = []
    def myFunc(x):
        if x in filtering:
            return False
        else:
            return True
    fit = filter (myFunc, comments)
    for x in fit:
        data.append(x)
    return data
data['Sentence'] = data['Sentence'].apply(stopword_removal)
data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Sentence,Sentiment
0,"[the, geosolutions, technology, will, leverage...",positive
1,"[esi, on, lows, down, 150, to, 250, bk, a, rea...",negative
2,"[for, the, last, quarter, of, 2010, componenta...",positive
3,"[according, to, the, finnish-russian, chamber,...",neutral
4,"[the, swedish, buyout, firm, has, sold, its, r...",neutral


In [ ]:
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
def stemming(comments):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comments:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean=[]
    d_clean = " ".join(do)
    print(d_clean)
    return d_clean

data['Sentence'] = data['Sentence'].apply(stemming)
data.to_csv('data_clean.csv', index=False)
data_clean = pd.read_csv('data_clean.csv', encoding='latin1')
data_clean.head()

the geosolutions technology will leverage benefon s gps solutions by providing location based search technology a communities platform location relevant multimedia content and a new and powerful commercial model
esi on lows down 150 to 250 bk a real possibility
for the last quarter of 2010 componenta s net sales doubled to eur131m from eur76m for the same period a year earlier while it moved to a zero pre-tax profit from a pre-tax loss of eur7m
according to the finnish-russian chamber of commerce all the major construction companies of finland are operating in russia
the swedish buyout firm has sold its remaining 224 percent stake almost eighteen months after taking the company public in finland
spy wouldn t be surprised to see a green close
shell s 70 billion bg deal meets shareholder skepticism
ssh communications security corp stock exchange release october 14 2008 at 245 pm the company updates its full year outlook and estimates its results to remain at loss for the full year
kone s

,Sentence,Sentiment
0,the geosolutions technology will leverage bene...,positive
1,esi on lows down 150 to 250 bk a real possibility,negative
2,for the last quarter of 2010 componenta s net ...,positive
3,according to the finnish-russian chamber of co...,neutral
4,the swedish buyout firm has sold its remaining...,neutral


In [ ]:
print(data_clean.columns)

Index(['Sentence', 'Sentiment'], dtype='object')


In [ ]:
data_clean = data_clean.astype({'Sentiment': 'category'})
data_clean = data_clean.astype({'Sentence': 'string'})

data_clean.dtypes

Sentence       string
Sentiment    category
dtype: object

In [ ]:
# proses TF-IDF dan
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(data_clean['Sentence'].astype('U'))
text_tf

<5842x12146 sparse matrix of type '<class 'numpy.float64'>'
	with 97174 stored elements in Compressed Sparse Row format>

In [ ]:
# proses TF-IDF
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

tvec = CountVectorizer()
X_cVec = tvec.fit_transform(data_clean['Sentence'].values.astype('U'))
print(X_cVec)

h_tfidf = TfidfTransformer()
x_tfidf = h_tfidf.fit_transform(X_cVec)
print(x_tfidf)

X = data_clean.Sentence
Y = data_clean.Sentiment

  (0, 10992)	1
  (0, 5414)	1
  (0, 10893)	2
  (0, 11914)	1
  (0, 6899)	1
  (0, 2431)	1
  (0, 5511)	1
  (0, 10282)	1
  (0, 2796)	1
  (0, 8900)	1
  (0, 7026)	2
  (0, 2339)	1
  (0, 9854)	1
  (0, 3284)	1
  (0, 8562)	1
  (0, 9280)	1
  (0, 7664)	1
  (0, 3454)	1
  (0, 1952)	2
  (0, 7800)	1
  (0, 8690)	1
  (0, 3262)	1
  (0, 7545)	1
  (1, 4514)	1
  (1, 8052)	1
  :	:
  (5840, 560)	1
  (5841, 1952)	1
  (5841, 5174)	1
  (5841, 5270)	1
  (5841, 6362)	1
  (5841, 5686)	1
  (5841, 5765)	1
  (5841, 7105)	1
  (5841, 5228)	1
  (5841, 1939)	1
  (5841, 9694)	1
  (5841, 10473)	2
  (5841, 2457)	1
  (5841, 8116)	1
  (5841, 1762)	1
  (5841, 3586)	1
  (5841, 5884)	1
  (5841, 7423)	1
  (5841, 11963)	1
  (5841, 6683)	1
  (5841, 8809)	1
  (5841, 6089)	1
  (5841, 6583)	1
  (5841, 6752)	1
  (5841, 10566)	1
  (0, 11914)	0.0867023718543157
  (0, 10992)	0.04274849531646452
  (0, 10893)	0.29139790726294024
  (0, 10282)	0.14481406256238066
  (0, 9854)	0.22431352058475024
  (0, 9280)	0.22431352058475024
  (0, 8900)	0.1900

In [ ]:
# splitting data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text_tf, data_clean['Sentiment'], test_size=0.2, random_state=42)

In [ ]:
# performa algoritma naive bayes
from sklearn.naive_bayes import MultinomialNB  # Fix typo in the import statement
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

clf = MultinomialNB().fit(X_train, y_train)  # Fix typo in the variable names
predicted = clf.predict(X_test)

print('MultinomialNB Accuracy:', accuracy_score(y_test, predicted))
print('MultinomialNB Precision:', precision_score(y_test, predicted, average='micro'))
print('MultinomialNB Recall:', recall_score(y_test, predicted, average='micro', pos_label='negative'))
print('MultinomialNB f1 score:', f1_score(y_test, predicted, average='micro', pos_label='negative'))

print(f'confusion matrix:\n{confusion_matrix(y_test, predicted)}')
print('================================================')
print(classification_report(y_test, predicted, zero_division=0))


MultinomialNB Accuracy: 0.6475620188195038
MultinomialNB Precision: 0.6475620188195038
MultinomialNB Recall: 0.6475620188195038
MultinomialNB f1 score: 0.6475620188195038
confusion matrix:
[[  7 128  40]
 [  1 612   9]
 [  0 234 138]]
              precision    recall  f1-score   support

    negative       0.88      0.04      0.08       175
     neutral       0.63      0.98      0.77       622
    positive       0.74      0.37      0.49       372

    accuracy                           0.65      1169
   macro avg       0.75      0.46      0.45      1169
weighted avg       0.70      0.65      0.58      1169



C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'negative') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'negative') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Splitting data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(text_tf, data_clean['Sentiment'], test_size=0.2, random_state=42)

# Membuat dan melatih model Naive Bayes
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Melakukan prediksi pada data uji
predicted = clf.predict(X_test)

# Menghitung dan mencetak metrik-metrik evaluasi
print('Accuracy:', accuracy_score(y_test, predicted))
print('Precision:', precision_score(y_test, predicted, average='micro', pos_label='negative'))
print('Recall:', recall_score(y_test, predicted, average='micro', pos_label='negative'))
print('F1 Score:', f1_score(y_test, predicted, average='micro', pos_label='negative'))

# Confusion matrix
print('Confusion Matrix:\n', confusion_matrix(y_test, predicted))

# Classification report
print('Classification Report:\n', classification_report(y_test, predicted))


Accuracy: 0.6475620188195038
Precision: 0.6475620188195038
Recall: 0.6475620188195038
F1 Score: 0.6475620188195038
Confusion Matrix:
 [[  7 128  40]
 [  1 612   9]
 [  0 234 138]]
Classification Report:
               precision    recall  f1-score   support

    negative       0.88      0.04      0.08       175
     neutral       0.63      0.98      0.77       622
    positive       0.74      0.37      0.49       372

    accuracy                           0.65      1169
   macro avg       0.75      0.46      0.45      1169
weighted avg       0.70      0.65      0.58      1169



C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'negative') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'negative') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'negative') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
